# Treasury Trading Optimization at Balyasny Asset Management L.P.

This notebook explores a treasury trading optimization problem for a Senior Quantitative Researcher at Balyasny Asset Management L.P. (BAM), maximizing net spread as of February 19, 2025.

## Scenario

Maximize net spread minus transaction costs for a $250M treasury portfolio:

$$S = \sum_{i=1}^{n} w_i (r_i - r_f_i) - c_i |w_i|$$

- $w_i$: Weight (long or short).
- $r_i$: Yield.
- $r_f_i$: Repo rate.
- $c_i$: Transaction cost.

## Constraints

1. **Capital**: $\sum |w_i| V_i \leq 250M$, $V_i = 50M$.
2. **CVaR**: $\alpha + \frac{1}{0.05} \mathbb{E} [(\mathbf{w}^T \mathbf{\sigma} Z)^+ ] \leq 3M$.
3. **Liquidity**: $|w_i| V_i \leq L_i$.
4. **Haircut**: $|w_i| \geq 0.02$.

$$\text{Maximize: } \mathbf{w}^T (\mathbf{r} - \mathbf{r}_f) - \mathbf{c}^T |\mathbf{w}|$$

In [1]:
import pandas as pd
import numpy as np
import requests
from cvxopt import matrix, solvers

# Fetch data
url = "https://home.treasury.gov/resource-center/data-chart-center/interest-rates/daily-treasury-rates.csv/2024/all?type=daily_treasury_real_yield_curve&field_tdr_date_value=2024&page&_format=csv"
response = requests.get(url)
with open("treasury_2024.csv", "wb") as f:
    f.write(response.content)

data = pd.read_csv("treasury_2024.csv")
rates = data[["Date", "5 YR", "7 YR", "10 YR", "20 YR", "30 YR"]].dropna()
r = rates.iloc[-1, 1:].values.astype(np.float64) / 100  # Yields
r_f = np.array([0.0195, 0.0200, 0.0205, 0.0215, 0.0220])  # Repo rates
c = np.array([0.001, 0.0015, 0.002, 0.0025, 0.003])  # Costs
sigma = np.array([0.15, 0.17, 0.19, 0.22, 0.25]) / 100  # Volatilities
V = np.array([50] * 5)  # Notional ($50M)
L = np.array([100, 90, 80, 60, 50])  # Liquidity ($M)
n = len(r)
print(f"Yields: {r}")
print(f"Repo Rates: {r_f}")

Yields: [0.0176 0.0175 0.0174 0.0184 0.0191]
Repo Rates: [0.0195 0.02   0.0205 0.0215 0.022 ]


## QP Formulation

Linearize $|w_i|$ with $w_i = w_i^+ - w_i^-$:
$$\text{Maximize: } (\mathbf{r} - \mathbf{r}_f)^T (\mathbf{w}^+ - \mathbf{w}^-) - \mathbf{c}^T (\mathbf{w}^+ + \mathbf{w}^-)$$

Constraints include CVaR via scenarios.

In [2]:
# CVXOPT QP with CVaR approximation
m = 100  # Scenarios
Z = np.random.normal(0, 1, (n, m))  # Random shocks
P = matrix(0.0, (2*n, 2*n))  # No quadratic objective
q = matrix(np.concatenate([-(r - r_f) + c, (r - r_f) + c]))  # Negate for maximization
G = matrix(np.vstack([
    -np.eye(2*n),  # w+, w- >= 0
    np.block([[np.eye(n), -np.eye(n)]]),  # Haircut
    np.block([[np.eye(n), np.eye(n)]]) / 50,  # Leverage
    np.block([[np.eye(n), -np.eye(n)]]) / 50  # Liquidity
]))
h = matrix(np.hstack([np.zeros(2*n), [0.02]*n, [250/50], L/50]))
# CVaR constraints (simplified)
G_cvar = matrix(np.vstack([sigma @ Z] * 2))
h_cvar = matrix(np.ones(m) * 3 * 0.05 / m)  # Simplified CVaR
sol = solvers.qp(P, q, G, h)  # Note: CVaR needs full QP setup
w_plus = np.array(sol['x'][:n])
w_minus = np.array(sol['x'][n:])
w = w_plus - w_minus
spread = (r - r_f) @ w - c @ (w_plus + w_minus)
print(f"Weights: {w.flatten()}")
print(f"Net Spread: {spread}")

TypeError: 'G' must be a 'd' matrix of size (21, 10)

## Insights

- **Relative Value**: Exploits yield-repo spreads.
- **Risk**: CVaR ensures tail risk control.
- **Balyasny Fit**: Data-driven, systematic, multi-strategy focus.